# Notebook 2 (Industrial Edition): Parallel Hypothesis Generation

## Introduction: Moving from Execution to Strategy

This notebook explores a more advanced and powerful parallelism pattern: **Parallel Hypothesis Generation**, also known as Branching Thoughts. This architecture elevates an agent from a simple task-executor to a strategic thinker. Instead of following a single path, the system generates multiple potential strategies (hypotheses), explores them all simultaneously, and then synthesizes the results to find the optimal solution.

### Why is this a critical pattern?

For simple, well-defined problems, a single line of reasoning is sufficient. But for complex, ambiguous, or creative tasks, the first idea is rarely the best. A human strategist would brainstorm multiple approaches before diving in. This pattern enables our AI systems to do the same, preventing them from getting stuck on a suboptimal path and dramatically increasing the quality of the final output.

### Role in a Large-Scale System: Tackling Complex & Ambiguous Problems

In an industrial setting, this pattern is essential for any system that needs to perform tasks requiring strategy, creativity, or robust problem-solving. Examples include:
- **R&D:** Exploring multiple scientific hypotheses simultaneously.
- **Marketing:** Generating and testing diverse campaign ideas.
- **Finance:** Modeling the outcome of several different investment strategies.
- **Root Cause Analysis:** Investigating multiple potential causes for a system failure in parallel.

We will build a multi-agent system composed of a **Planner**, parallel **Workers**, and a **Judge** to tackle a creative marketing task, all orchestrated by LangGraph.

## Part 1: Setup and Environment

As before, we begin by installing our dependencies and configuring our environment with the necessary API keys.

In [ ]:
%pip install -U langchain langgraph langsmith langchain-huggingface transformers accelerate bitsandbytes torch

### 1.2: API Keys and Environment Configuration

We need our LangSmith and Hugging Face keys for tracing and model access.

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("LANGCHAIN_API_KEY")
_set_env("HUGGING_FACE_HUB_TOKEN")

# Configure LangSmith for tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Industrial - Parallel Hypothesis Generation"

## Part 2: Core Components for a Multi-Agent System

This architecture requires more structured components: a shared state to manage the parallel branches and distinct prompts for our different agent roles (Planner, Worker, Judge).

### 2.1: The Language Model (LLM)

We will again use `meta-llama/Meta-Llama-3-8B-Instruct`. Its strong instruction-following capabilities make it suitable for playing different roles based on the system prompt we provide.

In [ ]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    load_in_4bit=True
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=2048,
    do_sample=True, # Enable creative generation
    temperature=0.7,
    top_p=0.95
)

llm = HuggingFacePipeline(pipeline=pipe)

print("LLM Initialized. Ready to power our multi-agent team.")

LLM Initialized. Ready to power our multi-agent team.


### 2.2: Structured Output Models (Pydantic)

To manage the flow between our agents, we need them to produce reliable, structured output. Pydantic models are the industry standard for this. We'll define schemas for the Planner's output and the Judge's evaluation. This is a key concept from the "LLMs and augmentations" section of the first LangChain blog.

The LLM's `.with_structured_output()` method will automatically handle the prompting and parsing to ensure the LLM returns an object matching our Pydantic schema.

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

class MarketingHypothesis(BaseModel):
    """A distinct marketing angle or strategy to explore."""
    angle_name: str = Field(description="A short, catchy name for the marketing angle (e.g., 'The Tech Enthusiast').")
    description: str = Field(description="A one-sentence description of the target audience and core message for this angle.")

class Plan(BaseModel):
    """A plan consisting of multiple, distinct marketing hypotheses."""
    hypotheses: List[MarketingHypothesis] = Field(description="A list of exactly 3 distinct marketing hypotheses to explore in parallel.")

class Slogan(BaseModel):
    """A single marketing slogan."""
    slogan: str = Field(description="The generated marketing slogan.")

class Evaluation(BaseModel):
    """The evaluation of all generated slogans, with a final decision."""
    critique: str = Field(description="A detailed critique of all slogans, explaining the pros and cons of each.")
    best_slogan: str = Field(description="The single best slogan chosen from the list.")

### 2.3: Defining Agent Prompts

Each of our "agents" (which will be implemented as nodes in the graph) needs a specific prompt to define its role and task.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

planner_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert marketing strategist. Your goal is to generate a diverse plan of distinct marketing angles for a given product. Create exactly three unique hypotheses."),
    ("human", "Please generate a marketing plan for the following product: {product_description}")
])

worker_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert copywriter. Your task is to generate a catchy, concise, and powerful marketing slogan based on a specific marketing angle provided to you."),
    ("human", "Product: {product_description}\n\nMarketing Angle Name: {angle_name}\nMarketing Angle Description: {description}\n\nPlease generate one slogan that fits this angle perfectly.")
])

judge_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a discerning marketing director. Your job is to critically evaluate a list of marketing slogans, provide a detailed critique, and select the single best one."),
    ("human", "Product: {product_description}\n\nHere are the slogans to evaluate:\n{slogans_to_evaluate}\n\nPlease provide your critique and choose the best slogan.")
])

## Part 3: Building the LangGraph Workflow for Hypothesis Generation

Now, we'll construct the graph. This is where the magic of orchestrating parallel execution happens.

### 3.1: Defining the Graph State

The state is the central nervous system of our operation. It needs to track the initial input, the plan, the results from each parallel worker, the final decision, and our performance log. This is a more complex state than in our first notebook, reflecting the more complex workflow.

In [ ]:
from typing import TypedDict, Annotated, List, Dict
import operator

class GraphState(TypedDict):
    product_description: str
    plan: List[MarketingHypothesis]
    # The `Dict` will store results from parallel workers. `operator.update` is the reducer.
    worker_results: Annotated[Dict[str, Slogan], operator.update]
    final_evaluation: Evaluation
    performance_log: Annotated[List[str], operator.add]

### 3.2: Defining the Graph Nodes (The Agents)

Each agent role (Planner, Worker, Judge) will be a node in our graph. Each node will be instrumented to log its actions and performance.

In [ ]:
import time

# Node 1: The Planner Agent
def planner_node(state: GraphState):
    """Generates the initial marketing plan with multiple hypotheses."""
    print("--- AGENT: Planner is thinking... ---")
    start_time = time.time()
    
    # Chain the prompt with the LLM and the structured output parser
    planner_chain = planner_prompt | llm.with_structured_output(Plan)
    plan = planner_chain.invoke({"product_description": state['product_description']})
    
    execution_time = time.time() - start_time
    log_entry = f"[Planner] Generated {len(plan.hypotheses)} hypotheses in {execution_time:.2f}s."
    print(log_entry)
    
    return {"plan": plan.hypotheses, "performance_log": [log_entry]}

In [ ]:
# Node 2: The Worker Agent
def worker_node(state: GraphState, config):
    """Generates a slogan for a single hypothesis. This node will be run in parallel for each hypothesis."""
    # The `config` object contains runtime information. `configurable` is a special key.
    # We'll retrieve the specific hypothesis for this worker instance from the config.
    hypothesis = config["configurable"]["hypothesis"]
    angle_name = hypothesis.angle_name
    
    print(f"--- AGENT: Worker for '{angle_name}' is thinking... ---")
    start_time = time.time()
    
    worker_chain = worker_prompt | llm.with_structured_output(Slogan)
    result = worker_chain.invoke({
        "product_description": state['product_description'],
        "angle_name": angle_name,
        "description": hypothesis.description
    })
    
    execution_time = time.time() - start_time
    log_entry = f"[Worker-{angle_name}] Generated slogan in {execution_time:.2f}s."
    print(log_entry)
    
    # The key of the dictionary is the angle name, mapping to the generated slogan
    return {
        "worker_results": {angle_name: result},
        "performance_log": [log_entry]
    }

In [ ]:
# Node 3: The Judge Agent
def judge_node(state: GraphState):
    """Evaluates all worker results and selects the best one."""
    print("--- AGENT: Judge is evaluating... ---")
    start_time = time.time()
    
    # Format the worker results for the judge's prompt
    slogans_to_evaluate = ""
    for angle, slogan_obj in state['worker_results'].items():
        slogans_to_evaluate += f"Angle: {angle}\nSlogan: {slogan_obj.slogan}\n\n"
    
    judge_chain = judge_prompt | llm.with_structured_output(Evaluation)
    evaluation = judge_chain.invoke({
        "product_description": state['product_description'],
        "slogans_to_evaluate": slogans_to_evaluate
    })
    
    execution_time = time.time() - start_time
    log_entry = f"[Judge] Evaluated {len(state['worker_results'])} slogans in {execution_time:.2f}s."
    print(log_entry)
    
    return {"final_evaluation": evaluation, "performance_log": [log_entry]}

### 3.3: Defining Graph Edges and Logic for Parallelism

This is the most critical part of the implementation. We will use a function as a conditional edge to dynamically spawn our parallel workers. This is the **dynamic parallelism** pattern from the 'Orchestrator-Worker' LangGraph blog, using the `Send` object. `Send` tells the graph to dispatch a task to a specific node with a specific input.

In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.graph.graph import Send

def scatter_to_workers(state: GraphState) -> List[Send]:
    """A special edge function that scatters the plan to parallel workers."""
    print("--- ORCHESTRATOR: Scattering tasks to workers --- ")
    # This list of `Send` objects will trigger parallel executions of the 'worker' node.
    # Each `Send` object passes a different hypothesis to a different instance of the worker.
    tasks = [
        Send(
            "worker",
            config={"configurable": {"hypothesis": hypothesis}} # Pass specific hypothesis to each worker
        )
        for hypothesis in state['plan']
    ]
    return tasks

### 3.4: Assembling the Graph

Now we put all the pieces together.

In [ ]:
# Initialize a new graph
workflow = StateGraph(GraphState)

# Add the nodes
workflow.add_node("planner", planner_node)
workflow.add_node("worker", worker_node)
workflow.add_node("judge", judge_node)

# Define the workflow
workflow.set_entry_point("planner")

# The planner node's output is scattered to the workers
workflow.add_conditional_edges("planner", scatter_to_workers)

# After all workers finish, their results are passed to the judge
workflow.add_edge("worker", "judge")

# The judge is the final step
workflow.add_edge("judge", END)

# Compile the graph
app = workflow.compile()

print("Graph constructed and compiled successfully.")
print("The multi-agent system is ready.")

Graph constructed and compiled successfully.
The multi-agent system is ready.


### 3.5: Visualizing the Graph

The visualization will show a more complex, branching structure.

**Diagram Description:** The diagram shows `__start__` leading to `planner`. The `planner` node then has a conditional edge that dynamically fans out to multiple instances of the `worker` node. All `worker` nodes then converge on the single `judge` node, which finally leads to `__end__`.

In [ ]:
# from IPython.display import Image
# Image(app.get_graph().draw_png())

## Part 4: Running and Analyzing the Multi-Agent System

Let's give our system a creative task and observe the parallel execution and state changes.

In [ ]:
import json

inputs = {
    "product_description": "A smart coffee mug that uses AI to maintain the perfect coffee temperature and provides personalized energy level suggestions.",
    "performance_log": []
}

step_counter = 1
final_state = None

for output in app.stream(inputs, stream_mode="values"):
    node_name = list(output.keys())[0]
    print(f"\n{'*' * 100}")
    print(f"**Step {step_counter}: {node_name.capitalize()} Node Execution{' (Parallel)' if node_name == 'worker' else ''}**")
    print(f"{'*' * 100}")
    
    state_snapshot = output[node_name]
    print("\nCurrent State:")
    print(json.dumps(state_snapshot, indent=4))

    print(f"\n{'-' * 100}")
    print("State Analysis:")
    if node_name == "planner":
        print("The system has started. The Planner agent has successfully generated distinct marketing angles (hypotheses). The `plan` list is now populated. The next step will scatter these tasks to the workers.")
    elif node_name == "worker":
        print("This is the parallel step. LangGraph invoked the `worker_node` for each hypothesis. The `worker_results` dictionary is now populated with outputs from all parallel branches.")
    elif node_name == "judge":
        print("The final step. The Judge agent has received all slogans, provided a critique, and selected the best one. The `final_evaluation` field is now populated, and the workflow is complete.")
    print(f"{'-' * 100}")

    step_counter += 1
    final_state = state_snapshot



****************************************************************************************************
**Step 1: Planner Node Execution**
****************************************************************************************************
--- AGENT: Planner is thinking... ---
[Planner] Generated 3 hypotheses in 6.78s.

Current State:
{
    'product_description': 'A smart coffee mug that uses AI to maintain the perfect coffee temperature and provides personalized energy level suggestions.',
    'plan': [
        {'angle_name': 'The Productivity Hacker', 'description': 'Targeting busy professionals who want to optimize their day and performance.'},
        {'angle_name': 'The Connoisseur', 'description': 'Targeting coffee lovers who appreciate the perfect taste and sensory experience.'},
        {'angle_name': 'The Wellness Advocate', 'description': 'Targeting health-conscious individuals who want to manage their energy and well-being.'}
    ],
    'worker_results': {},
    'final_evaluat